In [1]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime

In [2]:
# Set the limit for number of articles to download
LIMIT = 50

In [3]:
data = {}
data['newspapers'] = {}

In [4]:
# Loads the JSON files with news sites
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

In [5]:
count = 1

# Iterate through each news company
# the company is the name, the value is the dictionary of links
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data.
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['text'] = content.text
                article['author'] = content.authors
                newsPaper['articles'].append(article)
                if count % 10 == 0:
                    print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.
            if content.publish_date is None:
                print(count, " Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 10:
                    print("Too many noneType dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            article['author'] = content.authors
            newsPaper['articles'].append(article)
            if count % 10 == 0: 
                print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper


# Finally it saves the articles as a JSON-file.
try:
    fname = 'scraped_articles.json'
    print('saving articles . . . in {}'.format(fname))
    with open(fname, 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)

10 articles downloaded from newyorktimes_business , url:  https://www.nytimes.com/2018/03/23/business/trump-recession-forecast.html?partner=rss&emc=rss
20 articles downloaded from newyorktimes_business , url:  https://www.nytimes.com/video/business/100000005788184/what-bananas-tell-us-about-trade-wars.html?partner=rss&emc=rss
30 articles downloaded from newyorktimes_business , url:  https://www.nytimes.com/2018/03/23/obituaries/h-wayne-huizenga-entrepreneur-and-team-owner-is-dead-at-80.html?partner=rss&emc=rss
40 articles downloaded from newyorktimes_business , url:  https://www.nytimes.com/2018/03/23/travel/changes-at-the-airport.html?partner=rss&emc=rss
10 articles downloaded from newyorktimes_science , url:  https://www.nytimes.com/2018/03/23/health/right-to-try-drugs-fda.html?partner=rss&emc=rss
20 articles downloaded from newyorktimes_science , url:  https://www.nytimes.com/2018/03/21/health/breast-implants-lymphoma.html?partner=rss&emc=rss
30 articles downloaded from newyorktimes

30 articles downloaded from satprnews , url:  https://www.satprnews.com/2018/03/25/pigment-intermediate-market-opportunities-and-forecasts-2017-2025/
40 articles downloaded from satprnews , url:  https://www.satprnews.com/2018/03/25/pharmaceutical-solvent-market-will-reach-at-a-cagr-of-4-2-from-2016-2026/
50 articles downloaded from satprnews , url:  https://www.satprnews.com/2018/03/25/3-2-cagr-anticipated-for-bulk-terminals-market-during-2016-2026/
10 articles downloaded from insidertradings , url:  https://www.insidertradings.org/global-and-chinese-motorized-positioners-for-robots-industry-2018-market-research-report/
10 articles downloaded from highlandmirror , url:  https://www.highlandmirror.com/2018-03-20-global-ftir-spectrometer-market-2018-new-era-of-industry-swot-analysis-business-overview-and-forecast-2025
You must `download()` an article first!

continuing...
20 articles downloaded from highlandmirror , url:  https://www.highlandmirror.com/2018-03-20-global-diabetic-shoes-s